In [1]:
import onnx

In [2]:
model = onnx.load('onnx/model.onnx')
onnx.checker.check_model(model)

In [4]:
# 造一个新的输出
model1_output_tensor_type = model.graph.output[0].type.tensor_type
output_elem_type = model1_output_tensor_type.elem_type
make_output = onnx.helper.make_tensor_value_info(
                                'out_rgb',
                                output_elem_type,
                                [1,3,1024,1024]
                            )
# 删掉旧的输出，把新的输出加进去
model.graph.output.pop()
model.graph.output.append(make_output)

In [6]:
# 造一个新的输入
model1_input_tensor_type = model.graph.input[0].type.tensor_type
input_elem_type = model1_input_tensor_type.elem_type
make_input = onnx.helper.make_tensor_value_info(
                                'in_raw',
                                input_elem_type,
                                [1,4,512,512]
                            )
# 删掉旧的输入，把新的输入加进去
model.graph.input.pop()
model.graph.input.append(make_input)

In [19]:
# 将到数第二层的输出接到输出，再扔掉最后一层的permute
model.graph.node[-2].output[0] = 'out_rgb'
model.graph.node.pop()

input: "DepthToSpace:0"
output: "out_rgb:0"
name: "DepthToSpace__61"
op_type: "Transpose"
attribute {
  name: "perm"
  ints: 0
  ints: 2
  ints: 3
  ints: 1
  type: INTS
}

In [29]:
# 将第二个节点的输入接到输入，reverse之后把最后一个删了再reverse回来
model.graph.node[1].input[0] = 'in_raw'
model.graph.node.reverse()
model.graph.node.pop()
model.graph.node.reverse()

In [49]:
onnx.save(model, 'onnx/slove.onnx')